In [1]:
import pandas as pd
import re
import regex

In [2]:
data = pd.read_csv("../data/tawiki_pages_articles.csv")

In [3]:
# Extraction Regex

re_boxed_values = "\[[^\]^\[]*\]"
re_curlBoxed_values = "\{[^\{]*\}"
re_heading_values = "\=\=([^\=]*)\=\="
re_boxed_and_curls = "\[[^\]^\[]*\]\] \([^\(^\)]*\)"
re_categories = "\[\[பகுப்பு:([^\]]*)\]"
  

In [4]:
# Cleaning Regex

#set of regex rules to extract patterns

re_braces = "\([^\)]*\)"
re_curl_braces ="\{\{[^\}\{]*\}\}"
re_single_curl = "\{[^\}\{]*\}"
# re_curl_braces ="\{[^\}]*\}"
# re_double_curl_braces = "\{[^\{]*\{[^\}]*\}\}[^\}]*\}\}"
re_category = "\[\[பகுப்பு:[^\]]*\]"
re_header = "(\=\=[^\=]*\=\=)"
re_img = "\[\[படிமம்:.*\]\]"
re_ref1 = "<ref.*?/>"
re_ref2 = "<ref.*?</ref>"

In [5]:
def extract_nouns(text):
  # Extract nouns only Tamil
  text_r = text
  boxed = re.findall( r"\[\[(.*?)\]\]",text)
  nouns = []
  for box in boxed:
      if ':' not in box:
          if "|" in box:
              nouns.append(box.split("|")[0])
              text_r = text_r.replace(box,box.split("|")[1])
          else:
              nouns.append(box)
          
  # filter only tamil
  nouns = [noun for noun in nouns if regex.match(r"[\p{Tamil} .]",noun)]
  
  
  # return noun in replaced text
  return nouns , text_r

def parse_data(text,title):
  global boxed_values, curlBoxed_values, heading_values, boxed_and_curls, categories, all_nouns
  boxed_values.append({'title':title,'box':list(set(re.findall(re_boxed_values,text,re.M)))})
  curlBoxed_values.append({'title':title,'curl':list(set(re.findall(re_curlBoxed_values,text,re.M)))})
  heading_values.append({'title':title,'heading':list(set(re.findall(re_heading_values,text,re.M)))})
  boxed_and_curls.append({'title':title,'boxcurl':list(set(re.findall(re_boxed_and_curls,text,re.M)))})
  categories.append({'title':title,'categories':list(set(re.findall(re_categories,text,re.M)))})
  nouns , text_r = extract_nouns(text)
  all_nouns += nouns
  all_nouns = list(set(all_nouns))
  return text_r

def clean_data(text):
  text = re.sub(re_braces," ",text)
  text = re.sub(re_ref2," ",text)
  text = re.sub(re_ref1," ",text)
  text = re.sub(re_curl_braces," ",text)
  text = re.sub(re_curl_braces," ",text)
  text = re.sub(re_curl_braces," ",text)
  text = re.sub(re_single_curl," ",text)
  text = re.sub(re_category," ",text)
  text = re.sub(re_header," ",text)
  text ="".join(regex.findall("[\p{Tamil} \.]",text))
  text = re.sub("\s\s+" , " ", text)
  return text

In [6]:
section_rm= ['மேற்கோள்கள்','இவற்றையும் பார்க்கவும்','வெளி இணைப்புகள்','நூற்பட்டியல்','துணை நூல்கள்']

def remove_sections(text):
  text = re.sub(re_img,"",text)
  article = ""
  for para in text.split("\n\n"):
    if not any([section in para for section in section_rm]):
      article += para 
      
  return article

In [7]:
#combine all these into a single funciton

def extract_tawiki(text,title):
  if type(text)==str:
    text = remove_sections(text)
    text = parse_data(text,title)
    clean_text = clean_data(text)
    return clean_text
  else:
    return ""
  

In [8]:
#text data
tawiki_data= []


#Other useful information
boxed_values = []
curlBoxed_values = []
heading_values = []
boxed_and_curls = []
categories = []
all_nouns = []

In [9]:
extract_tawiki(data.text.iloc[300],data.title.iloc[0])

' செவ்வாய் சூரியக்குடும்பத்தில் உள்ள ஒரு கோள் ஆகும். இது சூரியனிலிருந்து நான்காவது கோளாக உள்ளது.இக்குடும்பத்தில் மிகச்சிறிய கோளான புதனுக்கு அடுத்ததாக இரண்டாவது சிறிய கோளாக செவ்வாய் இருக்கிறது. மேனாட்டினர் இக்கோளுக்கு போர்க்கடவுளின் பெயரைச் சூட்டியுள்ளனர். இதன் மேற்பரப்பில் காணப்படும் இரும்பு ஆக்சைடு இக்கோளைச் செந்நிறமாகக் காட்டுகிறது. இதனாலேயே இதற்குச் செவ்வாய் என்ற பெயர் ஏற்பட்டது. ஒரு புவிசார் கோளான இதன் மேற்பரப்பு சந்திரனில் உள்ளதுபோல் கிண்ணக் குழிகளையும் புவியில் உள்ளது போன்ற எரிமலைகள் பள்ளத்தாக்குகள் பாலைவனங்கள் பனிமூடிய துருவப் பகுதிகளையும் கொண்டது. செவ்வாயின் சுழற்சிக்காலமும் பருவ மாற்றங்களும் புவிக்கு உள்ளதைப் போன்றவையே. சூரிய மண்டலத்துள் மிக உயரமான ஒலிம்பசு மலையும் மிகப்பெரிய செங்குத்துப் பள்ளத்தாக்குகளுள் ஒன்றான மரினர் பள்ளத்தாக்கும் செவ்வாயிலேயே உள்ளன. ஆம் ஆண்டில் செவ்வாய்க்கு அண்மையாக மரினர் வெற்றிகரமாகப் பறந்து செல்லும்வரை செவ்வாய்க் கோளின் மேற்பரப்பில் நீர்ம நீர் இருக்கும் என நம்பினர். கோளின் துருவப் பகுதிகளுக்கு அருகில் குறித்த கால அடிப்படையில் மாற்றம் அடைகின்ற கறுப்பு வ

In [10]:
for i in range(data.shape[0]):
  tawiki_data.append(extract_tawiki(data.text.iloc[i],data.title.iloc[i]))

In [11]:
#data to be saved
tawiki_data_df = pd.DataFrame(tawiki_data,columns=['Content'])
boxed_values_df = pd.DataFrame(boxed_values)
#curl_boxed_values = pd.DataFrame(curlBoxed_values)
boxed_and_curls_df = pd.DataFrame(boxed_and_curls)
categories_df = pd.DataFrame(categories)

In [12]:
categories_df.head()

,title,categories
0,முதற் பக்கம்,"[இயங்குபடம் உள்ள கட்டுரைகள், செவ்வாய்க் கோள்| ..."
1,முதற் பக்கம்,[]
2,விக்கிப்பீடியா:கலந்துரையாடல்,[விக்கிப்பீடியா ஆலமரத்தடி]
3,விக்கிப்பீடியா:புதுப் பயனர் பக்கம்,[விக்கிப்பீடியா உதவி]
4,விக்கிப்பீடியா:விக்கிப்பீடியர்கள்,[விக்கிப்பீடியர்கள்]


Save all the results to disk.



In [13]:
tawiki_data_df.to_csv("../data/tawiki_data_df.csv",index=False)
boxed_values_df.to_csv("../data/boxed_value_df.csv",index =False)

boxed_and_curls_df.to_csv("../data/boxednCurls_df.csv",index=False)
categories_df.to_csv("../data/categories_df.csv",index=False)

In [14]:
pd.DataFrame({'nonuns':all_nouns}).to_csv("../data/all_nouns.csv",index=False)